<h1 style="text-align:center">Human Activity Recognition</h1>

In [1]:
# code to unzip a file
from zipfile import ZipFile
file_name = "HumanActivityRecognition.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("Done")

'\nfrom zipfile import ZipFile\nfile_name = "HumanActivityRecognition.zip"\n\nwith ZipFile(file_name,\'r\') as zip:\n  zip.extractall()\n  print("Done")\n'

In [0]:
# importing libraries
import pandas as pd
import numpy as np

In [0]:
# there are total 6 human activities which are treated as the class labels 
# we need to predict these class labels
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

In [ ]:
# function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])
    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

In [0]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [0]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [0]:
# function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# function to load the data
def load_signals(subset):
    signals_data = []
    for signal in SIGNALS:
        filename = f'HAR/UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 
    # transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [0]:
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'HAR/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]
    return pd.get_dummies(y).as_matrix()

In [0]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')
    return X_train, X_test, y_train, y_test

In [0]:
# importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [0]:
# configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [0]:
# import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [0]:
# importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [0]:
# initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [0]:
# function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [0]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [0]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


<h1>MODEL 1 :- 1 LSTM with 32 hidden layers and 'rmsprop' optimizer</h1>

In [0]:
# initiliazing the sequential model
model = Sequential()
# configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# adding a dropout layer
model.add(Dropout(0.5))
# adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [0]:
# compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
# training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Instructions for updating:
Use tf.cast instead.
Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 28s 4ms/step - loss: 1.3187 - acc: 0.4366 - val_loss: 1.1297 - val_acc: 0.4815
Epoch 2/30
7352/7352 [==============================] - 27s 4ms/step - loss: 0.9571 - acc: 0.5898 - val_loss: 0.8921 - val_acc: 0.5898
Epoch 3/30
7352/7352 [==============================] - 28s 4ms/step - loss: 0.7675 - acc: 0.6510 - val_loss: 0.7696 - val_acc: 0.6098
Epoch 4/30
7352/7352 [==============================] - 28s 4ms/step - loss: 0.6663 - acc: 0.6800 - val_loss: 0.7184 - val_acc: 0.6271
Epoch 5/30
7352/7352 [==============================] - 28s 4ms/step - loss: 0.6172 - acc: 0.7137 - val_loss: 0.8463 - val_acc: 0.6899
Epoch 6/30
7352/7352 [==============================] - 27s 4ms/step - loss: 0.5785 - acc: 0.7505 - val_loss: 0.6704 - val_acc: 0.7275
Epoch 7/30
7352/7352 [==============================] - 27s 4ms/step - loss: 0.5228 - acc: 0.7

In [0]:
# confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        27        0                   0   
SITTING                  2      375       113        0                   0   
STANDING                 0       54       474        4                   0   
WALKING                  0        8        10      467                   2   
WALKING_DOWNSTAIRS       0        0         0        0                 408   
WALKING_UPSTAIRS         0       12         7        5                  10   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            1  
STANDING                           0  
WALKING                            9  
WALKING_DOWNSTAIRS                12  
WALKING_UPSTAIRS                 437  


In [0]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 1s 338us/step


In [0]:
score

[0.3421370585869036, 0.9063454360366474]

<h1>MODEL 2 : 1 LSTM with 32 hidden layers and 'adam' optimizer</h1>

In [0]:
# initiliazing the sequential model
model = Sequential()
# configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# adding a dropout layer
model.add(Dropout(0.5))
# adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [0]:
# compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
# training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 29s 4ms/step - loss: 1.3789 - acc: 0.4159 - val_loss: 1.2244 - val_acc: 0.4723
Epoch 2/30
7352/7352 [==============================] - 27s 4ms/step - loss: 1.2241 - acc: 0.4512 - val_loss: 1.2673 - val_acc: 0.4164
Epoch 3/30
7352/7352 [==============================] - 28s 4ms/step - loss: 1.1998 - acc: 0.4557 - val_loss: 1.4244 - val_acc: 0.3610
Epoch 4/30
7352/7352 [==============================] - 27s 4ms/step - loss: 1.2836 - acc: 0.4115 - val_loss: 1.3771 - val_acc: 0.4018
Epoch 5/30
7352/7352 [==============================] - 27s 4ms/step - loss: 1.2467 - acc: 0.4285 - val_loss: 1.2943 - val_acc: 0.4143
Epoch 6/30
7352/7352 [==============================] - 27s 4ms/step - loss: 1.2174 - acc: 0.4343 - val_loss: 1.2652 - val_acc: 0.4751
Epoch 7/30
7352/7352 [==============================] - 27s 4ms/step - loss: 1.1803 - acc: 0.4653 - val_loss: 1.0857 - val_acc: 0.5263
Epoch 8

In [0]:
# confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0        0                   0   
SITTING                  1      405        78        0                   0   
STANDING                 0      103       427        2                   0   
WALKING                  0        0         0      464                  28   
WALKING_DOWNSTAIRS       0        0         0        2                 409   
WALKING_UPSTAIRS         0        0         1       21                  27   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                            7  
STANDING                           0  
WALKING                            4  
WALKING_DOWNSTAIRS                 9  
WALKING_UPSTAIRS                 422  


In [0]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 1s 340us/step


In [0]:
score

[0.3263602286097184, 0.8948082796063793]

<h1>MODEL 3 :- 1 LSTM with 64 hidden layers and 'rmsprop' optimizer</h1>

In [0]:
# initiliazing the sequential model
n_hidden = 64
model = Sequential()
# configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# adding a dropout layer
model.add(Dropout(0.5))
# adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 64)                18944     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 390       
Total params: 19,334
Trainable params: 19,334
Non-trainable params: 0
_________________________________________________________________


In [0]:
# compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
# training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 37s 5ms/step - loss: 1.2375 - acc: 0.4504 - val_loss: 1.1631 - val_acc: 0.4489
Epoch 2/30
7352/7352 [==============================] - 35s 5ms/step - loss: 0.9816 - acc: 0.5809 - val_loss: 0.9082 - val_acc: 0.6254
Epoch 3/30
7352/7352 [==============================] - 34s 5ms/step - loss: 0.8381 - acc: 0.6296 - val_loss: 0.9560 - val_acc: 0.6396
Epoch 4/30
7352/7352 [==============================] - 35s 5ms/step - loss: 0.8333 - acc: 0.6552 - val_loss: 0.7277 - val_acc: 0.7106
Epoch 5/30
7352/7352 [==============================] - 35s 5ms/step - loss: 0.6197 - acc: 0.7518 - val_loss: 0.7702 - val_acc: 0.7421
Epoch 6/30
7352/7352 [==============================] - 35s 5ms/step - loss: 0.4471 - acc: 0.8489 - val_loss: 0.5738 - val_acc: 0.8246
Epoch 7/30
7352/7352 [==============================] - 35s 5ms/step - loss: 0.3394 - acc: 0.8866 - val_loss: 0.4992 - val_acc: 0.8381
Epoch 8

In [0]:
# confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        25        0                   0   
SITTING                  1      416        74        0                   0   
STANDING                 0      100       430        2                   0   
WALKING                  0        0         0      445                   7   
WALKING_DOWNSTAIRS       0        0         0        0                 405   
WALKING_UPSTAIRS         0        2         0        2                   6   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             2  
SITTING                            0  
STANDING                           0  
WALKING                           44  
WALKING_DOWNSTAIRS                15  
WALKING_UPSTAIRS                 461  


In [0]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 2s 516us/step


In [0]:
score

[0.39258112489413843, 0.9049881235154394]

<h1>MODEL 4 :- 1 LSTM with 64 hidden layers and 'adam' optimizer</h1>

In [0]:
# initiliazing the sequential model
n_hidden = 64
model = Sequential()
# configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# adding a dropout layer
model.add(Dropout(0.5))
# adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 64)                18944     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 390       
Total params: 19,334
Trainable params: 19,334
Non-trainable params: 0
_________________________________________________________________


In [0]:
# compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
# training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 36s 5ms/step - loss: 1.3701 - acc: 0.4083 - val_loss: 1.2693 - val_acc: 0.4252
Epoch 2/30
7352/7352 [==============================] - 35s 5ms/step - loss: 1.1306 - acc: 0.4939 - val_loss: 1.1024 - val_acc: 0.4954
Epoch 3/30
7352/7352 [==============================] - 35s 5ms/step - loss: 1.1857 - acc: 0.4827 - val_loss: 1.1866 - val_acc: 0.5199
Epoch 4/30
7352/7352 [==============================] - 35s 5ms/step - loss: 1.2605 - acc: 0.4780 - val_loss: 1.5138 - val_acc: 0.3685
Epoch 5/30
7352/7352 [==============================] - 35s 5ms/step - loss: 1.3377 - acc: 0.3815 - val_loss: 1.3315 - val_acc: 0.3865
Epoch 6/30
7352/7352 [==============================] - 35s 5ms/step - loss: 1.3095 - acc: 0.4090 - val_loss: 1.2647 - val_acc: 0.3916
Epoch 7/30
7352/7352 [==============================] - 34s 5ms/step - loss: 1.1188 - acc: 0.5091 - val_loss: 1.0288 - val_acc: 0.5568
Epoch 8

In [0]:
# confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 507        0         0        1                   0   
SITTING                  0      393        78        4                   0   
STANDING                 0      108       402       10                   0   
WALKING                  0        0        15      194                   0   
WALKING_DOWNSTAIRS       0        0         4      199                   1   
WALKING_UPSTAIRS         0        3         7      174                   0   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            29  
SITTING                           16  
STANDING                          12  
WALKING                          287  
WALKING_DOWNSTAIRS               216  
WALKING_UPSTAIRS                 287  
Pred                LAYING  SITTING  STANDING  WALKING  WALKING

In [0]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 1s 492us/step


In [0]:
score

[0.7947044912984826, 0.6053613844587716]

[0.7947044912984826, 0.6053613844587716]

<h1>Conclusion :-</h1>

In [3]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Model", "Number of Layers","Optimizer", "Train log loss","Train Accuracy","Test log loss","Test Accuracy"]
x.add_row(["LSTM","32","rmsprop","0.2026","93.77","0.3421","90.63"])
x.add_row(["LSTM","32","adam","0.2263","93.77","0.3264","89.48"])
x.add_row(["LSTM","64","rmsprop","0.1310","95.42","0.3926","90.50"])
x.add_row(["LSTM","64","adam","0.7936","62.57","0.7947","60.54"])
print (x)

+-------+------------------+-----------+----------------+----------------+---------------+---------------+
| Model | Number of Layers | Optimizer | Train log loss | Train Accuracy | Test log loss | Test Accuracy |
+-------+------------------+-----------+----------------+----------------+---------------+---------------+
|  LSTM |        32        |  rmsprop  |     0.2026     |     93.77      |     0.3421    |     90.63     |
|  LSTM |        32        |    adam   |     0.2263     |     93.77      |     0.3264    |     89.48     |
|  LSTM |        64        |  rmsprop  |     0.1310     |     95.42      |     0.3926    |     90.50     |
|  LSTM |        64        |    adam   |     0.7936     |     62.57      |     0.7947    |     60.54     |
+-------+------------------+-----------+----------------+----------------+---------------+---------------+


<h2>Steps Followed :-</h2>

- Load the raw data.
- Split the data into train and test.
- Try out different LSTM models with different number of hidden layers and different optimizers.
- Draw confusion matrix.
- Print score of the predicted data.
- At the end in conclusion section comapre each LSTM model.